In [1]:
import joblib
import pandas as pd
import numpy as np
import cv2
import os

from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
import warnings

# Suppress UserWarning related to feature names
warnings.filterwarnings("ignore", category=UserWarning, message="X does not have valid feature names*")



In [2]:
current_directory = os.getcwd()
current_directory

'C:\\Users\\dujsi\\Desktop\\Tracker-main\\Tracker-main'

In [3]:
path = f'{current_directory}\\Camel'
path

'C:\\Users\\dujsi\\Desktop\\Tracker-main\\Tracker-main\\Camel'

[7, 11, 15, 19, 20, 17, 13, 9, 23, 18, 3, 4]

In [4]:
model_path = f'{current_directory}\\Pretrained-models'

# Load the scaler and imputer from files
scaler_a = joblib.load(f'{model_path}/scaler_a.pkl')
imputer_a = joblib.load(f'{model_path}/imputer_a.pkl')
model_a = joblib.load(f'{model_path}/model_a.pkl')

# Load the scaler and imputer from files
scaler_b = joblib.load(f'{model_path}/scaler_b.pkl')
imputer_b = joblib.load(f'{model_path}/imputer_b.pkl')
model_b = joblib.load(f'{model_path}/model_b.pkl')

scaler_best =joblib.load(f'{model_path}/scaler_best.pkl')
imputer_best =joblib.load(f'{model_path}/imputer_best.pkl')
model1_best = joblib.load(f'{model_path}/model_best1.pkl')
model2_best = joblib.load(f'{model_path}/model_best2.pkl')

In [6]:
def extract_image_features(frame, frame2, x, y, width, height, frame_features):
    # frame is visible
    # frame2 is IR
    frame_copy = frame.copy()
    frame2_copy = frame2.copy()
    roi = frame_copy[y:y+height, x:x+width]
    roi2 = frame2_copy[y:y+height, x:x+width]

    frame_gray = cv2.cvtColor(frame_copy, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.cvtColor(frame2_copy, cv2.COLOR_BGR2GRAY)
    roi_gray = frame_gray[y:y+height, x:x+width]
    roi2_gray = frame2_gray[y:y+height, x:x+width]

    rois = [(roi_gray, roi),
            (roi2_gray, roi2)]

    if frame_features:
            rois = [(frame_gray, frame_copy),
            (frame2_gray, frame2_copy),
            (roi_gray, roi),
            (roi2_gray, roi2)]

    features = []

    for x in rois:
        gray_roi = x[0]
        roi = x[1]

        # Convert the cropped region to BGR
        b, g, r = cv2.split(roi)

        mean_intensity_b = np.mean(b)
        mean_intensity_g = np.mean(g)
        mean_intensity_r = np.mean(r)

        # Compute median intensity directly from pixel values
        median_intensity_b = np.median(b)
        median_intensity_g = np.median(g)
        median_intensity_r = np.median(r)

        # Compute mode intensity directly from pixel values
        mode_intensity_b = np.argmax(np.bincount(b.flatten()))
        mode_intensity_g = np.argmax(np.bincount(g.flatten()))
        mode_intensity_r = np.argmax(np.bincount(r.flatten()))

        # Compute standard deviation directly from pixel values
        std_deviation_b = np.std(b)
        std_deviation_g = np.std(g)
        std_deviation_r = np.std(r)

        # Compute intensity statistics
        mean_intensity = np.mean(gray_roi)
        variance_intensity = np.var(gray_roi)
        skewness_intensity = np.mean((gray_roi - mean_intensity) ** 3) / np.power(variance_intensity, 1.5)
        kurtosis_intensity = np.mean((gray_roi - mean_intensity) ** 4) / np.power(variance_intensity, 2) - 3

        area = width*height

        # Include all variables in the features list
        features += [mean_intensity_b, mean_intensity_g, mean_intensity_r,
                median_intensity_b, median_intensity_g, median_intensity_r,
                mode_intensity_b, mode_intensity_g, mode_intensity_r,
                std_deviation_b, std_deviation_g, std_deviation_r,
                mean_intensity, variance_intensity, skewness_intensity, kurtosis_intensity, area]

    # Return all features
    return features

In [7]:
def eval_sample(features, scaler, imputer, model):
    # Apply imputation
    features_imputed = imputer.transform([features])

    # Apply scaling
    features_scaled = scaler.transform(features_imputed)

    # Use the model to make predictions
    prediction = model.predict(features_scaled)

    return prediction

In [8]:
def read_bbox_data_from_dataframe(dataframe):
    bbox_data = {}
    latest_frames = {}
    for index, row in dataframe.iterrows():
        frame = int(row['frame_id'])
        track_id = int(row['track_id'])
        class_id = int(row['class_id'])
        bbox = [int(row['absolute_gt_x']), int(row['absolute_gt_y']), int(row['absolute_gt_width']), int(row['absolute_gt_height'])]
        bbox_data.setdefault(frame, []).append((track_id, bbox))
        latest_frames[track_id] = max(frame, latest_frames.get(track_id, 0))  # Update the latest frame for the track_id
    return bbox_data, latest_frames

In [9]:
def read_absolute_truth(abs_file):
    df = pd.read_csv(abs_file, delimiter='\t', names= ['frame_id', 'track_id','class_id', 'absolute_gt_x', 'absolute_gt_y', 'absolute_gt_width', 'absolute_gt_height'], dtype={'frame_id': int, 'track_id': int,'class': int, 'x': float, 'y': float, 'width': float, 'height': float}, on_bad_lines='skip')
    return df

In [10]:
def is_bbox_valid(bbox, frame):
    image_width, image_height, _ = frame.shape
    x_min, y_min, width, height = bbox
    x_max = x_min + width
    y_max = y_min + height
    return 0 <= x_min < x_max <= image_width and 0 <= y_min < y_max <= image_height and width > 0 and height > 0

In [11]:
def adjust_bounding_box(frame, roi):
    height, width, _ = frame.shape
    height -= 1
    width -= 1
    x, y, w, h = roi

    # Adjust x coordinate if it's less than 0
    if x < 0:
        w += x  # Decrease width
        x = 0   # Set x to 0

    # Adjust width if it exceeds the frame width
    if x + w > width:
        w = width - x

    # Adjust y coordinate if it's less than 0
    if y < 0:
        h += y  # Decrease height
        y = 0   # Set y to 0

    # Adjust height if it exceeds the frame height
    if y + h > height:
        h = height - y

    # Ensure width and height are at least 2
    w = max(2, w)
    h = max(2, h)

        # Adjust width if it exceeds the frame width
    if x + w > width:
        x = width - w

    # Adjust height if it exceeds the frame height
    if y + h > height:
        y = height - h


    return (x, y, w, h)

In [12]:
import numpy as np

def ir_better(frame_rgb, frame_ir, bbox):
    frame_rgb = frame_rgb.copy()
    frame_ir = frame_ir.copy()

    #calculate contrast
    # Crop the selected ROI from the RGB image
    roi_rgb = frame_rgb[int(bbox[1]):int(bbox[1]+bbox[3]),
                    int(bbox[0]):int(bbox[0]+bbox[2])]

    roi_ir = frame_ir[int(bbox[1]):int(bbox[1]+bbox[3]),
                    int(bbox[0]):int(bbox[0]+bbox[2])]

    # Convert ROI from RGB to grayscale for contrast calculation
    roi_gray_rgb = cv2.cvtColor(roi_rgb, cv2.COLOR_BGR2GRAY)
    # Convert ROI from IR to grayscale for contrast calculation
    roi_gray_ir = cv2.cvtColor(roi_ir, cv2.COLOR_BGR2GRAY)

    std_deviation_roi_rgb = np.std(roi_gray_rgb)
    std_deviation_roi_ir = np.std(roi_gray_ir)

    return std_deviation_roi_ir > std_deviation_roi_rgb


In [13]:
def is_mostly_dark_in_region(image, bbox, threshold=100, brightness_threshold = 20):
    # Extract the region of interest (ROI) from the image using the bounding box
    x, y, w, h = bbox

    gray_roi = image[y:y+h, x:x+w]

    gray_roi = cv2.cvtColor(gray_roi, cv2.COLOR_BGR2GRAY)

    # Compute the average brightness of the ROI
    average_brightness = cv2.mean(gray_roi)[0]

    # Compute the difference between the highest and lowest point in the grayscale ROI
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(gray_roi)
    difference = max_val - min_val

    # Check if the average brightness is below the threshold
    if average_brightness < threshold and difference < brightness_threshold:
        return True
    else:
        return False

In [14]:
def init_tracker_6(tracker, bbox, frame, frame2, seq):
    ir_is_better = ir_better(frame, frame2 ,tuple(bbox))

    if ir_is_better:
        tracker.init(frame2, tuple(bbox))
    else:
        tracker.init(frame, tuple(bbox))

    return {'tracker': tracker, 'last_bbox': bbox,'ir_better': ir_is_better}

def update_tracker_6(data, frame, frame2, seq):
    frame = frame.copy()
    frame2 = frame2.copy()

    bbox = data['last_bbox']
    data['ir_better'] = ir_better(frame, frame2 ,tuple(bbox))

    if data['ir_better']:
        frame, frame2 = frame2, frame

    return data['tracker'].update(frame)

In [15]:
def init_tracker_1_a(tracker, bbox, frame, frame2, seq):
    tracker.init(frame, tuple(bbox))
    return {'tracker': tracker, 'last_bbox': bbox}

def update_tracker_1_a(data, frame, frame2, seq):
    return data['tracker'].update(frame)

def init_tracker_1_b(tracker, bbox, frame, frame2, seq):
    tracker.init(frame2, tuple(bbox))
    return {'tracker': tracker, 'last_bbox': bbox}

def update_tracker_1_b(data, frame, frame2, seq):
    return data['tracker'].update(frame2)


In [16]:
def init_tracker_2_a(tracker, bbox, frame, frame2, seq):
    tracker.init(frame, tuple(bbox))
    return {'tracker': tracker, 'last_bbox': bbox}

def update_tracker_2_a(data, frame, frame2, seq):
    data['tracker'].update(frame2)
    return data['tracker'].update(frame)

def init_tracker_2_b(tracker, bbox, frame, frame2, seq):
    tracker.init(frame2, tuple(bbox))
    return {'tracker': tracker, 'last_bbox': bbox}

def update_tracker_2_b(data, frame, frame2, seq):
    data['tracker'].update(frame)
    return data['tracker'].update(frame2)

In [17]:
def init_tracker_3_a(tracker, bbox, frame, frame2, seq):
    tracker.init(frame, tuple(bbox))
    return {'tracker': tracker, 'last_bbox': bbox, 'lost_count': 0}

def update_tracker_3_a(data, frame, frame2, seq):
    success, bbox = data['tracker'].update(frame)
    if not success:
      data['lost_count'] +=1
      if data['lost_count'] > 3:
          success, bbox = data['tracker'].update(frame2)
    else:
        data['lost_count'] = 0

    return success, bbox

def init_tracker_3_b(tracker, bbox, frame, frame2, seq):
    tracker.init(frame2, tuple(bbox))
    return {'tracker': tracker, 'last_bbox': bbox, 'lost_count': 0}

def update_tracker_3_b(data, frame, frame2, seq):
    success, bbox = data['tracker'].update(frame2)
    if not success:
      data['lost_count'] +=1
      if data['lost_count'] > 3:
          success, bbox = data['tracker'].update(frame)
    else:
        data['lost_count'] = 0

    return success, bbox

In [18]:
def init_tracker_4(tracker, bbox, frame, frame2, seq):
    tracker.init(frame2, tuple(bbox))
    return {'tracker': tracker, 'last_bbox': bbox, 'frame_counter': 0}

def update_tracker_4(data, frame, frame2, seq):
    if data['frame_counter'] % 2 == 0:
        frame, frame2 = frame2.copy(), frame.copy()

    return data['tracker'].update(frame)

In [19]:
def init_tracker_5(tracker, bbox, frame, frame2, seq):
    ir_is_better = is_mostly_dark_in_region(frame, bbox)

    if ir_is_better:
        tracker.init(frame2, tuple(bbox))
    else:
        tracker.init(frame, tuple(bbox))

    return {'tracker': tracker, 'last_bbox': bbox,'ir_better': ir_is_better}

def update_tracker_5(data, frame, frame2, seq):
    frame = frame.copy()
    frame2 = frame2.copy()

    bbox = data['last_bbox']
    data['ir_better'] = is_mostly_dark_in_region(frame, bbox)

    if data['ir_better']:
        frame, frame2 = frame2, frame

    return data['tracker'].update(frame)


In [20]:
def init_tracker_7_a(tracker, bbox, frame, frame2, seq):
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = True)
    ir_is_better = eval_sample(features, scaler_a, imputer_a, model_a)

    if ir_is_better:
        tracker.init(frame2, tuple(bbox))
    else:
        tracker.init(frame, tuple(bbox))

    return {'tracker': tracker, 'last_bbox': bbox,'ir_better': ir_is_better, 'lost_count': 0}

def update_tracker_7_a(data, frame, frame2, seq):
    frame = frame.copy()
    frame2 = frame2.copy()

    if data['ir_better']:
        frame, frame2 = frame2, frame

    success, bbox = data['tracker'].update(frame)

    if not success:
        data['lost_count'] +=1
        if data['lost_count'] > 3:
            success, bbox = data['tracker'].update(frame2)
    else:
        data['lost_count'] = 0

    return success, bbox

def init_tracker_7_b(tracker, bbox, frame, frame2, seq):
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = False)
    ir_is_better = eval_sample(features, scaler_b, imputer_b, model_b)

    if ir_is_better:
        tracker.init(frame2, tuple(bbox))
    else:
        tracker.init(frame, tuple(bbox))

    return {'tracker': tracker, 'last_bbox': bbox,'ir_better': ir_is_better, 'lost_count': 0}

def update_tracker_7_b(data, frame, frame2, seq):
    frame = frame.copy()
    frame2 = frame2.copy()

    if data['ir_better']:
        frame, frame2 = frame2, frame

    success, bbox = data['tracker'].update(frame)

    if not success:
        data['lost_count'] +=1
        if data['lost_count'] > 3:
            success, bbox = data['tracker'].update(frame2)
    else:
        data['lost_count'] = 0

    return success, bbox

def init_tracker_7_best(tracker, bbox, frame, frame2, seq):
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = False)
    if seq in [7, 11, 15, 19, 20, 17, 13, 9, 23, 18, 3, 4]:
      model = model2_best
    else:
      model = model1_best
    ir_is_better = eval_sample(features, scaler_best, imputer_best, model)

    if ir_is_better:
        tracker.init(frame2, tuple(bbox))
    else:
        tracker.init(frame, tuple(bbox))

    return {'tracker': tracker, 'last_bbox': bbox,'ir_better': ir_is_better, 'lost_count': 0}

def update_tracker_7_best(data, frame, frame2, seq):
    frame = frame.copy()
    frame2 = frame2.copy()

    if data['ir_better']:
        frame, frame2 = frame2, frame

    success, bbox = data['tracker'].update(frame)

    if not success:
        data['lost_count'] +=1
        if data['lost_count'] > 3:
            success, bbox = data['tracker'].update(frame2)
    else:
        data['lost_count'] = 0

    return success, bbox

In [21]:
def init_tracker_8_a(tracker, bbox, frame, frame2, seq):
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = True)
    ir_is_better = eval_sample(features, scaler_a, imputer_a, model_a)

    if ir_is_better:
        tracker.init(frame2, tuple(bbox))
    else:
        tracker.init(frame, tuple(bbox))

    return {'tracker': tracker, 'last_bbox': bbox,'ir_better': ir_is_better}

def update_tracker_8_a(data, frame, frame2, seq):
    frame = frame.copy()
    frame2 = frame2.copy()

    bbox = data['last_bbox']
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = True)
    data['ir_better'] = eval_sample(features, scaler_a, imputer_a, model_a)

    if data['ir_better']:
        frame, frame2 = frame2, frame

    return data['tracker'].update(frame)

def init_tracker_8_b(tracker, bbox, frame, frame2, seq):
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = False)
    ir_is_better = eval_sample(features, scaler_b, imputer_b, model_b)

    if ir_is_better:
        tracker.init(frame2, tuple(bbox))
    else:
        tracker.init(frame, tuple(bbox))

    return {'tracker': tracker, 'last_bbox': bbox,'ir_better': ir_is_better}

def update_tracker_8_b(data, frame, frame2, seq):
    frame = frame.copy()
    frame2 = frame2.copy()

    bbox = data['last_bbox']
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = False)
    data['ir_better'] = eval_sample(features, scaler_b, imputer_b, model_b)

    if data['ir_better']:
        frame, frame2 = frame2, frame

    return data['tracker'].update(frame)

def init_tracker_8_best(tracker, bbox, frame, frame2, seq):
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = False)
    if seq in [7, 11, 15, 19, 20, 17, 13, 9, 23, 18, 3, 4]:
      model = model2_best
    else:
      model = model1_best
    ir_is_better = eval_sample(features, scaler_best, imputer_best, model)

    if ir_is_better:
        tracker.init(frame2, tuple(bbox))
    else:
        tracker.init(frame, tuple(bbox))

    return {'tracker': tracker, 'last_bbox': bbox,'ir_better': ir_is_better}

def update_tracker_8_best(data, frame, frame2, seq):
    frame = frame.copy()
    frame2 = frame2.copy()

    bbox = data['last_bbox']
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = False)
    if seq in [7, 11, 15, 19, 20, 17, 13, 9, 23, 18, 3, 4]:
      model = model2_best
    else:
      model = model1_best
    data['ir_better'] = eval_sample(features, scaler_best, imputer_best, model)


    if data['ir_better']:
        frame, frame2 = frame2, frame

    return data['tracker'].update(frame)

In [22]:
def init_tracker_9_a(tracker, bbox, frame, frame2, seq):
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = True)
    ir_is_better = eval_sample(features, scaler_a, imputer_a, model_a)

    if ir_is_better:
        tracker.init(frame2, tuple(bbox))
    else:
        tracker.init(frame, tuple(bbox))

    return {'tracker': tracker, 'last_bbox': bbox,'ir_better': ir_is_better, 'switch_count': 0}

def update_tracker_9_a(data, frame, frame2, seq):
    frame = frame.copy()
    frame2 = frame2.copy()

    bbox = data['last_bbox']
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = True)
    ir_better = eval_sample(features, scaler_a, imputer_a, model_a)
    if data['ir_better'] != ir_better:
        data['switch_count'] += 1
        if data['switch_count'] > 3:
            data['ir_better'] = ir_better
            data['switch_count'] = 0
    else:
        data['switch_count'] = 0

    if data['ir_better']:
        frame, frame2 = frame2, frame

    success, bbox = data['tracker'].update(frame)

    return data['tracker'].update(frame)

def init_tracker_9_b(tracker, bbox, frame, frame2, seq):
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = False)
    ir_is_better = eval_sample(features, scaler_b, imputer_b, model_b)

    if ir_is_better:
        tracker.init(frame2, tuple(bbox))
    else:
        tracker.init(frame, tuple(bbox))

    return {'tracker': tracker, 'last_bbox': bbox,'ir_better': ir_is_better, 'switch_count': 0}

def update_tracker_9_b(data, frame, frame2, seq):
    frame = frame.copy()
    frame2 = frame2.copy()

    bbox = data['last_bbox']
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = False)
    ir_better = eval_sample(features, scaler_b, imputer_b, model_b)
    if data['ir_better'] != ir_better:
        data['switch_count'] += 1
        if data['switch_count'] > 3:
            data['ir_better'] = ir_better
            data['switch_count'] = 0
    else:
        data['switch_count'] = 0

    if data['ir_better']:
        frame, frame2 = frame2, frame

    success, bbox = data['tracker'].update(frame)

    return data['tracker'].update(frame)

def init_tracker_9_best(tracker, bbox, frame, frame2, seq):
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = False)
    if seq in [7, 11, 15, 19, 20, 17, 13, 9, 23, 18, 3, 4]:
      model = model2_best
    else:
      model = model1_best
    ir_is_better = eval_sample(features, scaler_best, imputer_best, model)

    if ir_is_better:
        tracker.init(frame2, tuple(bbox))
    else:
        tracker.init(frame, tuple(bbox))

    return {'tracker': tracker, 'last_bbox': bbox,'ir_better': ir_is_better, 'switch_count': 0}

def update_tracker_9_best(data, frame, frame2, seq):
    frame = frame.copy()
    frame2 = frame2.copy()

    bbox = data['last_bbox']
    x, y, w, h = bbox
    features = extract_image_features(frame, frame2, x, y, w, h, frame_features = False)
    if seq in [7, 11, 15, 19, 20, 17, 13, 9, 23, 18, 3, 4]:
      model = model2_best
    else:
      model = model1_best
    ir_is_better = eval_sample(features, scaler_best, imputer_best, model)

    if data['ir_better'] != ir_better:
        data['switch_count'] += 1
        if data['switch_count'] > 3:
            data['ir_better'] = ir_better
            data['switch_count'] = 0
    else:
        data['switch_count'] = 0

    if data['ir_better']:
        frame, frame2 = frame2, frame

    success, bbox = data['tracker'].update(frame)

    return data['tracker'].update(frame)

In [23]:
# Map the function numbers and versions to the actual functions
init_tracker_functions = {
    (1, 'VIS'): init_tracker_1_a,
    (1, 'IR'): init_tracker_1_b,
    (2, 'VIS'): init_tracker_2_a,
    (2, 'IR'): init_tracker_2_b,
    (3, 'VIS'): init_tracker_3_a,
    (3, 'IR'): init_tracker_3_b,
    (4, None): init_tracker_4,
    (5, None): init_tracker_5,
    (6, None): init_tracker_6,
    (7, 'A'): init_tracker_7_a,
    (7, 'B'): init_tracker_7_b,
    (7, 'BEST'): init_tracker_7_best,
    (8, 'A'): init_tracker_8_a,
    (8, 'B'): init_tracker_8_b,
    (8, 'BEST'): init_tracker_8_best,
    (9, 'A'): init_tracker_9_a,
    (9, 'B'): init_tracker_9_b,
    (9, 'BEST'): init_tracker_9_best,
}

update_tracker_functions = {
    (1, 'VIS'): update_tracker_1_a,
    (1, 'IR'): update_tracker_1_b,
    (2, 'VIS'): update_tracker_2_a,
    (2, 'IR'): update_tracker_2_b,
    (3, 'VIS'): update_tracker_3_a,
    (3, 'IR'): update_tracker_3_b,
    (4, None): update_tracker_4,
    (5, None): update_tracker_5,
    (6, None): update_tracker_6,
    (7, 'A'): update_tracker_7_a,
    (7, 'B'): update_tracker_7_b,
    (7, 'BEST'): update_tracker_7_best,
    (8, 'A'): update_tracker_8_a,
    (8, 'B'): update_tracker_8_b,
    (8, 'BEST'): update_tracker_8_best,
    (9, 'A'): update_tracker_9_a,
    (9, 'B'): update_tracker_9_b,
    (9, 'BEST'): update_tracker_9_best,
}

# Create a list of keys from init_tracker_functions
init_tracker_keys = list(init_tracker_functions.keys())

In [24]:
init_tracker_keys

[(1, 'VIS'),
 (1, 'IR'),
 (2, 'VIS'),
 (2, 'IR'),
 (3, 'VIS'),
 (3, 'IR'),
 (4, None),
 (5, None),
 (6, None),
 (7, 'A'),
 (7, 'B'),
 (7, 'BEST'),
 (8, 'A'),
 (8, 'B'),
 (8, 'BEST'),
 (9, 'A'),
 (9, 'B'),
 (9, 'BEST')]

In [25]:
def annotate(vid_in_vis, vid_in_ir, annotation, annotation_file_out, tracker_number, seq, tracker_version=None):
    # Get the appropriate init and update functions based on tracker_number and tracker_version
    init_tracker = init_tracker_functions[(tracker_number, tracker_version)]
    update_tracker = update_tracker_functions[(tracker_number, tracker_version)]

    trackers = {}
    # Process frames and initialize trackers
    cap = cv2.VideoCapture(vid_in_vis)
    cap2 = cv2.VideoCapture(vid_in_ir)

    annotation_df = read_absolute_truth(annotation)
    bbox_data, latest_frames = read_bbox_data_from_dataframe(annotation_df)

    with open(annotation_file_out, 'w') as file:
        while cap.isOpened() and cap2.isOpened():
            ret, frame = cap.read()
            ret2, frame2 = cap2.read()
            if not ret or not ret2:
                break

            frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

            # Initialize trackers at the earliest frame where track_id appears
            if frame_number in bbox_data:
                for track_id, bbox in bbox_data[frame_number]:
                    if track_id not in trackers:
                        tracker = cv2.TrackerCSRT_create()
                        bbox = adjust_bounding_box(frame2, bbox)

                        trackers[track_id] = init_tracker(tracker, bbox, frame, frame2, seq)

            # Update trackers and draw bounding boxes
            for track_id, data in trackers.items():

                success, bbox = update_tracker(data, frame, frame2, seq)

                if success and is_bbox_valid(bbox, frame):
                    x, y, w, h = bbox
                    file.write(f"{frame_number} {track_id} {-1} {x} {y} {w} {h}\n")
                    data['last_bbox'] = bbox


            for track_id, value in latest_frames.items():
                if value == frame_number:
                    # Delete the tracker associated with the current track ID
                    del trackers[track_id]

            # Press 'q' to quit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break

    # Release resources
    cap.release()
    cap2.release()
    cv2.destroyAllWindows()

In [26]:
#7;6

In [32]:
import os
import shutil
from tqdm import tqdm

def clean_folders(path):
    for i in tqdm(range(1, 31)):
        # Skip specific sequences
        if i in [22, 24, 16, 14, 12]:
            continue
        
        # Define required filenames
        required_files = {
            f'Visual-seq{i}.mp4',
            f'IR-seq{i}.mp4',
            f'Seq{i}-Abs.txt'
        }
        
        # Target directory for current sequence
        seq_folder = os.path.join(path, f'seq-{i}')
        
        # Skip if the folder doesn't exist
        if not os.path.exists(seq_folder):
            continue
        
        # Iterate through files and folders in the sequence folder
        for filename in os.listdir(seq_folder):
            file_path = os.path.join(seq_folder, filename)
            
            # Remove files not in required list
            if os.path.isfile(file_path) and filename not in required_files:
                os.remove(file_path)
                print(f"Deleted file: {file_path}")
            
            # Remove directories that are not required
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
                print(f"Deleted folder: {file_path}")

clean_folders(path)

 23%|███████████████████▎                                                               | 7/30 [00:00<00:00, 45.22it/s]

Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-1\gt
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-1\IR
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-1\Visual
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-2\gt
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-2\IR
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-2\Visual
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-3\gt
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-3\IR
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-3\Visual
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-4\gt
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-4\IR
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-4\Visual
Deleted folder: C:\Users

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 79.75it/s]

Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-8\Visual
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-9\gt
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-9\IR
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-9\Visual
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-10\gt
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-10\IR
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-10\Visual
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-11\gt
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-11\IR
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-11\Visual
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-13\gt
Deleted folder: C:\Users\dujsi\Desktop\Tracker-main\Tracker-main\Camel\seq-13\IR
Deleted folder: 

In [29]:
for i in tqdm(range(1, 31)):
    if i in [22,24,16,14,12]:
        continue
    for (tracker_number, tracker_version) in tqdm(init_tracker_keys):
        annotate(f'{path}/seq-{i}/Visual-seq{i}.mp4',
                f'{path}/seq-{i}/IR-seq{i}.mp4',
                f'{path}/seq-{i}/Seq{i}-Abs.txt',
                f'{path}/results/seq-{i}/Seq{i}-{tracker_number}-{tracker_version}-Out.txt',
                tracker_number,
                i,
                tracker_version,
                )

  0%|                                                                                           | 0/30 [00:04<?, ?it/s]


KeyboardInterrupt: 